In [1]:
#импорт необходимых библиотек

import numpy as np
import pandas as pd
import os

from datetime import datetime
from datetime import timedelta

In [2]:
dataset = pd.read_csv(r'/Users/kirill/Downloads/archive/dataset.csv', index_col=0, header=[0, 1]).sort_index(axis=1)

In [3]:
dataset.head()

A                                      AAL         \
                     close   high      low     open   volume  close   high   
timestamp                                                                    
2017-09-11 09:30:00    NaN    NaN      NaN      NaN      NaN  44.01  44.05   
2017-09-11 09:31:00  65.46  65.50  65.4100  65.5000  29852.0  44.25  44.25   
2017-09-11 09:32:00  65.66  65.66  65.4604  65.4604   3435.0  44.27  44.32   
2017-09-11 09:33:00  65.69  65.70  65.6200  65.6700    700.0  44.34  44.46   
2017-09-11 09:34:00  65.88  65.88  65.6800  65.6900   2814.0  44.24  44.36   

                                              ...   ZION                       \
                         low   open   volume  ...  close   high    low   open   
timestamp                                     ...                               
2017-09-11 09:30:00  44.0100  44.01  39049.0  ...  42.04  42.05  42.04  42.05   
2017-09-11 09:31:00  44.0000  44.01  32620.0  ...  42.24  42.54  42.01  42.06   
2017-09-11 09:32:00  44.2200  44.25  22411.0  ...  42.48  42.48  42.09  42.12   
2017-09-11 09:33:00  44.2504  44.28  45320.0  ...  42.20  42.49  42.18  42.49   
2017-09-11 09:34:00  44.2200  44.34  17614.0  ...  42.38  42.38  42.23  42.28   

                                ZTS                                  
                      volume  close   high     low    open   volume  
timestamp                                                            
2017-09-11 09:30:00  26933.0    NaN    NaN     NaN     NaN      NaN  
2017-09-11 09:31:00  39292.0  65.11  65.39  64.960  65.330  38144.0  
2017-09-11 09:32:00   9683.0  65.12  65.23  65.075  65.115   4390.0  
2017-09-11 09:33:00   7844.0    NaN    NaN     NaN     NaN      NaN  
2017-09-11 09:34:00   6472.0  64.97  65.18  64.970  65.120   5515.0  

[5 rows x 2510 columns]

In [4]:
stocks = []
for el in list(dataset.columns):
    stocks.append(el[0])
    
stocks = list(set(stocks))

In [5]:


date = '2017-10-11'
df = dataset
i = 'A'



x = pd.to_datetime(df[i].reset_index()['timestamp']).dt.strftime('%Y-%m-%d')
df = df[i].reset_index(drop=False)
df['index'] = x
df = df.set_index('index')
del df['timestamp']


finish = str(datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d'))
start = str((datetime.strptime(date, '%Y-%m-%d') - timedelta(days=54*7)).strftime('%Y-%m-%d'))

df = df[start:finish]

In [6]:
def get_metrics(dataset, date, stocks):

    
    def get_boards(df):
        df['Upside potential'] = df[['low', 'high']].cummax(axis=1)['high'].cummax() / df['close']
        df['Downside protection'] = 1 - df[['low', 'high']].cummin(axis=1)['high'].cummin() / df['close']
        return(df)

    def get_rsi(df):
        up = df['close'].diff().clip(lower=0).expanding().mean()
        down = (df['close'].diff().clip(upper=0)*(-1)).expanding().mean()
        df['RSI'] = 100 - 100 / ( 1 + (up / down).replace(np.nan, 0) )
        return(df)

    drr = {}
    
    for i in stocks:
        df = dataset

        x = pd.to_datetime(df[i].reset_index()['timestamp']).dt.strftime('%Y-%m-%d')
        df = df[i].reset_index(drop=False)
        df['index'] = x
        df = df.set_index('index')
        del df['timestamp']

        finish = str(datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d'))
        start = str((datetime.strptime(date, '%Y-%m-%d') - timedelta(days=54*7)).strftime('%Y-%m-%d'))

        df = df[start:finish]
        
        df = get_rsi(get_boards(dataset[list(set(stocks))[0]]))[['Upside potential', 'Downside protection', 'RSI']]
    
        drr[i] = df
    return(drr)

In [7]:
c1 = get_metrics(dataset, date = '2017-10-11', stocks=['A'])
c2 = get_metrics(dataset, date = '2017-10-11', stocks=['AAP'])

/var/folders/rh/7rjcc_rs68n6wjb41vhqg3dh0000gn/T/ipykernel_20379/215899826.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Upside potential'] = df[['low', 'high']].cummax(axis=1)['high'].cummax() / df['close']
/var/folders/rh/7rjcc_rs68n6wjb41vhqg3dh0000gn/T/ipykernel_20379/215899826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Downside protection'] = 1 - df[['low', 'high']].cummin(axis=1)['high'].cummin() / df['close']
/var/folders/rh/7rjcc_rs68n6wjb41vhqg3dh0000gn/T/ipykernel_20379/2158

In [8]:
c1

{'A':                      Upside potential  Downside protection         RSI
 timestamp                                                             
 2017-09-11 09:30:00               NaN                  NaN    0.000000
 2017-09-11 09:31:00          1.000611             0.000764    0.000000
 2017-09-11 09:32:00          1.000000             0.003807  100.000000
 2017-09-11 09:33:00          1.000152             0.004262  100.000000
 2017-09-11 09:34:00          1.000000             0.007134  100.000000
 ...                               ...                  ...         ...
 2018-02-16 15:55:00          1.039861             0.116534   49.725009
 2018-02-16 15:56:00          1.040655             0.115860   49.722335
 2018-02-16 15:57:00          1.040799             0.115737   49.721849
 2018-02-16 15:58:00          1.041522             0.115123   49.719418
 2018-02-16 15:59:00          1.042115             0.114619   49.717425
 
 [43146 rows x 3 columns]}

# Второй вариант через rolling

In [17]:
new_ind = list(dataset.index)

for i in range(len(new_ind)):
    new_ind[i] = pd.Timestamp(new_ind[i])
    
dataset.index = new_ind

In [48]:
dataset['A']['close'].rolling('364d')

/Users/kirill/Library/Python/3.9/lib/python/site-packages/IPython/lib/pretty.py:778: FutureWarning: win_type will no longer return 'freq' in a future version. Check the type of self.window instead.
  output = repr(obj)


Rolling [window=364d,min_periods=1,center=False,win_type=freq,axis=0,method=single]

In [226]:
def get_metrics(df, per='378d', min_p=1):
    
    def get_bounds(st, p=per, m=min_p):
        up = st[['low', 'high']].rolling(window=p, min_periods=m).max().cummax()['high'] / st['close']
        down = st[['low', 'high']].rolling(window=p, min_periods=m).min().cummin()['high'] / st['close']
        return(list(up), list(down))
    
    def get_rsi(st, p=per, m=min_p):
        up = st['close'].diff().clip(lower=0).rolling(window='378d', min_periods=1).mean()
        down = (st['close'].diff().clip(upper=0)*(-1)).rolling(window='378d', min_periods=1).mean()
        rsi = 100 - 100 / ( 1 + (up / down).replace(np.nan, 0) )
        return(list(rsi))
    
    def decart(s1,s2):
        return([(a,b) for a in s1 for b in s2])
    
    def get_index(df):
        return(df[list(df.columns)[0]].index)
    
    def get_unique_numbers(numbers):
        unique = []
        for number in numbers:
            if number not in unique:
                unique.append(number)
        return(unique)
        
    
    stocks = []
    for el in list(dataset.columns):
        stocks.append(el[0])
    stocks = get_unique_numbers(stocks)
    metrics = ['Upside potential', 'Downside potential', 'RSI']
    
    data = []
    for i in stocks:
        x,y = get_bounds(df[i])
        z = get_rsi(df[i])
        data.append(x)
        data.append(y)
        data.append(z)
    data = np.matrix(data).T
    
    tuples = decart(stocks, metrics)
    
    itog = pd.DataFrame(data, index=get_index(dataset), columns=pd.MultiIndex.from_tuples(tuples))
    
    return(itog)

In [227]:
get_metrics(dataset)

A                                 \
                    Upside potential Downside potential         RSI   
2017-09-11 09:30:00              NaN                NaN    0.000000   
2017-09-11 09:31:00         1.000611           1.000611    0.000000   
2017-09-11 09:32:00         1.000000           0.997563  100.000000   
2017-09-11 09:33:00         1.000152           0.997108  100.000000   
2017-09-11 09:34:00         1.000000           0.994232  100.000000   
...                              ...                ...         ...   
2018-02-16 15:55:00         1.039861           0.884021   49.725009   
2018-02-16 15:56:00         1.040655           0.884695   49.722335   
2018-02-16 15:57:00         1.040799           0.884818   49.721849   
2018-02-16 15:58:00         1.041522           0.885433   49.719418   
2018-02-16 15:59:00         1.042115           0.885937   49.717425   

                                 AAL                                 \
                    Upside potential Downside potential         RSI   
2017-09-11 09:30:00         1.000909           1.000909    0.000000   
2017-09-11 09:31:00         1.000000           0.995480  100.000000   
2017-09-11 09:32:00         1.001129           0.995030  100.000000   
2017-09-11 09:33:00         1.002706           0.993460  100.000000   
2017-09-11 09:34:00         1.004973           0.995705   76.744186   
...                              ...                ...         ...   
2018-02-16 15:55:00         1.141422           0.851043   50.350084   
2018-02-16 15:56:00         1.141201           0.850879   50.350497   
2018-02-16 15:57:00         1.141643           0.851208   50.349660   
2018-02-16 15:58:00         1.142747           0.852031   50.347568   
2018-02-16 15:59:00         1.144296           0.853186   50.344640   

                                 AAP                                 \
                    Upside potential Downside potential         RSI   
2017-09-11 09:30:00              NaN                NaN    0.000000   
2017-09-11 09:31:00         1.005184           1.005184    0.000000   
2017-09-11 09:32:00         1.000000           0.999894  100.000000   
2017-09-11 09:33:00              NaN                NaN  100.000000   
2017-09-11 09:34:00         1.004283           0.999522  100.000000   
...                              ...                ...         ...   
2018-02-16 15:55:00         1.148818           0.731850   50.430284   
2018-02-16 15:56:00         1.148605           0.731714   50.430641   
2018-02-16 15:57:00         1.148392           0.731578   50.430999   
2018-02-16 15:58:00         1.149244           0.732121   50.429544   
2018-02-16 15:59:00         1.149911           0.732546   50.428407   

                                AAPL  ...         YUM              ZBH  \
                    Upside potential  ...         RSI Upside potential   
2017-09-11 09:30:00         1.000311  ...    0.000000         1.000000   
2017-09-11 09:31:00         1.001558  ...  100.000000         1.004301   
2017-09-11 09:32:00         1.002745  ...  100.000000         1.000086   
2017-09-11 09:33:00         1.003809  ...   88.888889         1.004128   
2017-09-11 09:34:00         1.002995  ...   61.538462         1.003956   
...                              ...  ...         ...              ...   
2018-02-16 15:55:00         1.041703  ...   49.987404         1.070590   
2018-02-16 15:56:00         1.041823  ...   49.986076         1.070767   
2018-02-16 15:57:00         1.042487  ...   49.986519         1.071033   
2018-02-16 15:58:00         1.043272  ...   49.986076         1.071211   
2018-02-16 15:59:00         1.043816  ...   49.984307         1.071832   

                                                               ZION  \
                    Downside potential         RSI Upside potential   
2017-09-11 09:30:00           1.000000    0.000000         1.000238   
2017-09-11 09:31:00           1.000000    0.000000         1.007102   
2017-09-11 09:32